In [15]:
# Dependencies
#from config import (census_key, gkey)
import gmaps
import numpy as np
import pandas as pd
import requests
import time
#from us import states
from scipy.stats import linregress
from matplotlib import pyplot as plt

In [16]:
#import baby name data
baby_name_original = pd.read_csv(
    "../Data/StateNames.csv", dtype="object", encoding="utf-8")
#display result
baby_name_original.head()

,Id,Name,Year,Gender,State,Count
0,1,Mary,1910,F,AK,14
1,2,Annie,1910,F,AK,12
2,3,Anna,1910,F,AK,10
3,4,Margaret,1910,F,AK,8
4,5,Helen,1910,F,AK,7


In [18]:
#Q2 How many names are used by both genders? More or less over time? More or less prevalent by region? KL
#Child Gender vs Name Percent Match

#The unique names from 1910-2014
namecount= baby_name_original['Name'].nunique()
namecount

30274

In [19]:
# The names used by each gender (female has more diverse name selections/options than male)
name= baby_name_original.loc[:,['Name','Gender']]
name

gender= name.groupby(['Gender']).nunique()['Name']
gender

Gender
F    20031
M    13139
Name: Name, dtype: int64

In [20]:
#The progression of numbers of unique names in each year

year= baby_name_original.loc[:,['Name','Year','Gender']]
year

gender= year.groupby(['Gender','Year']).nunique()['Name']
gender

Gender  Year
F       1910    1083
        1911    1066
        1912    1261
        1913    1350
        1914    1549
                ... 
M       2010    4404
        2011    4412
        2012    4366
        2013    4324
        2014    4377
Name: Name, Length: 210, dtype: int64

In [21]:
# The progression of numbers of unique names by state in gender

region= baby_name_original.loc[:,['Name','Gender','State']]
region

gender= region.groupby(['Gender','State']).nunique()['Name']
gender

Gender  State
F       AK         928
        AL        4231
        AR        3094
        AZ        3562
        CA       12727
                 ...  
M       VT         576
        WA        2412
        WI        2228
        WV        1345
        WY         651
Name: Name, Length: 102, dtype: int64